In [ ]:
pip install linearmodels

     |████████████████████████████████| 1.5 MB 30.3 MB/s 
     |████████████████████████████████| 9.8 MB 27.3 MB/s 
  Attempting uninstall: statsmodels
    Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


# Welcome
This is material for the **Panel Data** chapter in Scott Cunningham's book, [Causal Inference: The Mixtape](https://mixtape.scunning.com/).

In [ ]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from linearmodels import PanelOLS, PooledOLS
from IPython.display import display, HTML
def pretty_print(df):
    return display( HTML( df.to_html().replace("\\n","<br>") ) )

# Prepare data

In [18]:
#Labels of regressors
labdict={'unsafe':'Unprotected sex with client of any kind',
         'llength':'Ln(Length)',
         'reg':'Client was a Regular',
         'age_cl':'Age of Client',
         'asq_cl':'Age of Client Squared',
         'appearance_cl':'Client Attractiveness (Scale of 1 to 10)',
         'provider_second':'Second Provider Involved',
         'asian_cl':'Asian Client',
         'black_cl':'Black Client',
         'hispanic_cl':'Hispanic Client',
         'othrace_cl':'Other Ethnicity Client',
         'hot':'Met Client in Hotel',
         'massage_cl':'Gave Client a Massage',
         'age':'Age of provider',
         'asq':'Age of provider squared',
         'bmi':'Body Mass Index',
         'hispanic':'Hispanic',
         'black':'Black',
         'other':'Other',
         'asian':'Asian',
         'schooling':'Imputed Years of Schooling',
         'cohab':'Cohabitating (living with a partner) but unmarried',
         'married':'Currently married and living with your spouse',
         'divorced':'Divorced and not remarried',
         'separated':'Married but not currently living with your spouse'}

#List of regressors
regvars=list(labdict.keys())

#List of time-invariant regressors
tfixedvars=['age', 'asq', 'bmi', 'hispanic', 'black', 'other', 'asian', 'schooling', 'cohab', 'married', 'divorced', 'separated']

#List of time-variant regressors
tvars=[var for var in regvars if var not in tfixedvars]

#Dependent variable
depvar=['lnw']

In [ ]:
# Load data: 
df=pd.read_stata("https://raw.github.com/scunning1975/mixtape/master/sasp_panel.dta")
# Drop missing data: 
df=df.dropna()
# Keep balanced data (4 observations for each id)
df=df[df.groupby('id')['session'].transform('count')==4]
# Transform indicator
df.provider_second=df.provider_second.map({'1. No':0,'2. Yes':1}).astype('int64')

#Generate demeaned dataframe
dmdf=df[['id','session']].copy()
dmdf[regvars+depvar]=df[regvars+depvar]-df.groupby('id')[regvars+depvar].transform('mean')

#Declare panel setup:
df=df.set_index(['id','session'])
dmdf=dmdf.set_index(['id','session'])

# Regressions

## POLS

In [33]:
reg1= PooledOLS(df.lnw, df[regvars].assign(const=1).rename(columns=labdict)).fit(cov_type='clustered',cluster_entity=True)
reg1

Dep. Variable:,lnw,R-squared:,0.3026
Estimator:,PooledOLS,R-squared (Between):,0.2299
No. Observations:,1028,R-squared (Within):,0.4398
Date:,"Thu, Nov 18 2021",R-squared (Overall):,0.3026
Time:,11:56:04,Log-likelihood,-570.00
Cov. Estimator:,Clustered,,
,,F-statistic:,17.394
Entities:,257,P-value,0.0000
Avg Obs:,4.0000,Distribution:,"F(25,1002)"
Min Obs:,4.0000,,
Max Obs:,4.0000,F-statistic (robust):,7.2016


## Fixed Effects

In [32]:
reg2=PanelOLS(df.lnw,df[tvars].rename(columns=labdict)
              ,entity_effects=True).fit(cov_type='clustered', cluster_entity=True,debiased=True)
reg2

Dep. Variable:,lnw,R-squared:,0.5160
Estimator:,PanelOLS,R-squared (Between):,-0.7005
No. Observations:,1028,R-squared (Within):,0.5160
Date:,"Thu, Nov 18 2021",R-squared (Overall):,-0.6970
Time:,11:55:58,Log-likelihood,162.25
Cov. Estimator:,Clustered,,
,,F-statistic:,62.153
Entities:,257,P-value,0.0000
Avg Obs:,4.0000,Distribution:,"F(13,758)"
Min Obs:,4.0000,,
Max Obs:,4.0000,F-statistic (robust):,32.911


## Demeaned OLS

In [31]:
reg3= PooledOLS(dmdf.lnw, dmdf[tvars].assign(const=1).rename(columns=labdict)).fit(cov_type='clustered',
                                                                                   cluster_entity=True,debiased=True)
reg3.summary

Dep. Variable:,lnw,R-squared:,0.5160
Estimator:,PooledOLS,R-squared (Between):,-0.1863
No. Observations:,1028,R-squared (Within):,0.5160
Date:,"Thu, Nov 18 2021",R-squared (Overall):,0.5160
Time:,11:55:49,Log-likelihood,162.25
Cov. Estimator:,Clustered,,
,,F-statistic:,83.143
Entities:,257,P-value,0.0000
Avg Obs:,4.0000,Distribution:,"F(13,1014)"
Min Obs:,4.0000,,
Max Obs:,4.0000,F-statistic (robust):,32.879


## Put them together

In [118]:
def readreg(res,sig_levels=[0.01,0.05,0.1],bnum=3,senum=3):
  """
  Function to extract information from regression results
  res: input a linearmodels.panel.results.PanelResults object
  sig_levels: significance levels (list)
  bnum: numer of digits for parameters
  senum: number of digits for standard errors
  """
  # Extract stats
  stats=[str(cell).strip() for r  in res.summary.tables[0] for cell in r]
  stats={stats[i]:stats[i+1] for i in range(0, len(stats)-1,2)}
  # Extract estimates
  T=pd.DataFrame(res.summary.tables[1])
  T.columns=['var','b','se','tstat','pval','low','up']
  T=T[1:].astype('string')
  T[['b','se','pval']]=T[['b','se','pval']].astype('float',copy=False)
  #  Create stars
  T['star']=''
  for level in sorted(sig_levels):
    T['star']=T.star+np.where(T.pval<=level,"*","")
  T['out']=T.b.map(lambda x: str(round(x,bnum)))+T.star +"\n("+T.se.map(lambda x: str(round(x,senum)))+")"
  #Add stats
  T=T.append(pd.DataFrame({'var': ['No. Observations'],'out':[stats['No. Observations:']]}))
  return T.set_index('var')[['out']]


In [119]:
res=pd.concat([readreg(reg) for reg in [reg1,reg2,reg3]],axis=1).fillna('(-)')
res.columns=['POLS','FE','Demeaned FE']
pretty_print(res)

,POLS,FE,Demeaned FE
Unprotected sex with client of any kind,0.013(0.042),0.051*(0.028),0.051*(0.028)
Ln(Length),-0.308***(0.041),-0.434***(0.024),-0.434***(0.024)
Client was a Regular,-0.047(0.033),-0.037**(0.019),-0.037**(0.019)
Age of Client,-0.001(0.01),0.002(0.007),0.002(0.007)
Age of Client Squared,0.0(0.0),-0.0(0.0),-0.0(0.0)
Client Attractiveness (Scale of 1 to 10),0.02**(0.01),0.006(0.006),0.006(0.006)
Second Provider Involved,0.055(0.075),0.113*(0.06),0.113*(0.06)
Asian Client,-0.013(0.052),-0.01(0.034),-0.01(0.034)
Black Client,0.092(0.069),0.026(0.042),0.026(0.042)
Hispanic Client,0.052(0.073),-0.062(0.052),-0.062(0.052)


# QUESTIONS
- Interpret the effect of natural log of session length on the natural log of hourly wage. Describe the economic theory that might explain this relationship? (HINT: Consider the role that supplier fixed versus variable costs may have on the hourly wage.)
- Becker described discrimination in terms of taste based. This meant that social interactions with people of the other race were factors into marginal cost. Given that these persist, what does this imply about the effect that competition is having on discrimination?
- Hamermesh and Biddle suggest that beauty is valued on the market. Describe some reasons why there is no effect on client beauty once we use the within estimators?
- What other interesting results did you find in this analysis? Which ones surprised you and which ones were intuitive and why?